In [1]:
"""
Since this script redistributes vehicle counts from the subarea extraction, it
ignores walk, bike, transit, and school bus trips (from the Daysim trips list)

Note that there are slight discrepancies in the number of trips between the
Daysim trips list and Cube/CHAMP trip list.
"""

import polars as pl

In [5]:
daysim_trips_filepath = r"X:\Projects\DTX\CaltrainValidation\s8_2019_Base\daysim\abm_output1\_trip_2.dat"
subarea_extraction_od_filepath = r"Q:\Service Bureau\Data Requests\20240201_Kittelson_SFMTA_ValenciaCorridor\new_study_boundaries\subarea_extraction\2-intermediate_outputs-py\subarea_extraction-OD-PM.csv"
out_filepath = r"Q:\Service Bureau\Data Requests\20240201_Kittelson_SFMTA_ValenciaCorridor\new_study_boundaries\subarea-maz-vehicle-OD-PM.csv"
subarea_tazs = [
    140,
    144,
    156,
    157,
    158,
    165,
    176,
    183,
    185,
    191,
    204,
    205,
    221,
    223,
    230,
    232,
    236,
]
origin_in_subarea = pl.col("otaz").is_in(subarea_tazs)
destination_in_subarea = pl.col("dtaz").is_in(subarea_tazs)
# PM (deptm is in minutes since midnight)
time_period_filter = (pl.col("deptm") >= 930) & (pl.col("deptm") <= 1109)
# external station IDs were set to be <0
external_to_external = (pl.col("otaz") < 0) & (pl.col("dtaz") < 0)

In [3]:
daysim_trips = (
    pl.scan_csv(daysim_trips_filepath, separator="\t")
    .filter(
        (origin_in_subarea | destination_in_subarea)
        & time_period_filter
        &
        # only keep sov, hov2, hov3+, tnc; drop walk, bike, transit, school_bus
        pl.col("mode").is_in((3, 4, 5, 9))
    )
    .with_columns(
        # only look at vehicle trips because
        # the subarea extraction only outputs vehicle trips
        # pl.when(pl.col("mode") == 1).then(pl.lit("walk"))
        # .when(pl.col("mode") == 2).then(pl.lit("bike"))
        pl.when(pl.col("mode") == 3)
        .then(pl.lit("sov"))
        .when(pl.col("mode") == 4)
        .then(pl.lit("hov2"))
        .when(pl.col("mode") == 5)
        .then(pl.lit("hov3+"))
        # .when((pl.col("mode") == 6) & (pl.col("pathtype") == 3))
        # .then(pl.lit("transit-local_bus"))
        # .when((pl.col("mode") == 6) & (pl.col("pathtype") == 4))
        # .then(pl.lit("transit-light_rail"))
        # .when((pl.col("mode") == 6) & (pl.col("pathtype") == 5))
        # .then(pl.lit("transit-premium_bus"))
        # .when((pl.col("mode") == 6) & (pl.col("pathtype") == 6))
        # .then(pl.lit("transit-commuter_rail"))
        # .when((pl.col("mode") == 6) & (pl.col("pathtype") == 7))
        # .then(pl.lit("transit-ferry"))
        # .when(pl.col("mode") == 8).then(pl.lit("school_bus"))
        .when(pl.col("mode") == 9)
        .then(pl.lit("tnc"))
        .alias("mode")
    )
    .select("opcl", "dpcl", "otaz", "dtaz", "mode")
).collect()
# daysim_trips.write_csv(f"{out_filepath_root}-daysim_filtered.csv")

In [6]:
subarea_extraction_od = pl.read_csv(subarea_extraction_od_filepath)

In [7]:
external_to_external_od = subarea_extraction_od.filter(
    external_to_external
).select(
    pl.col("otaz").alias("omaz"),
    pl.col("dtaz").alias("dmaz"),
    pl.col("DA").fill_null(0),
    pl.col("SR2").fill_null(0),
    pl.col("SR3").fill_null(0),
    # TNC: TNC vehicle-trips
    (pl.col("TNC1") + pl.col("TNC2") + pl.col("TNC3"))
    .fill_null(0)
    .alias("TNC"),
    pl.col("TRK").fill_null(0),
    pl.col("COM").fill_null(0),
)

In [8]:
# TODO clean up internal/external duplication of code/logic


def calculate_maz_external_od(daysim_trips, subarea_extraction_od):
    exit_subarea = origin_in_subarea & ~destination_in_subarea
    enter_subarea = ~origin_in_subarea & destination_in_subarea
    daysim_exit_trips = daysim_trips.filter(exit_subarea)
    daysim_enter_trips = daysim_trips.filter(enter_subarea)
    subarea_extraction_exit_od = subarea_extraction_od.filter(exit_subarea)
    subarea_extraction_enter_od = subarea_extraction_od.filter(enter_subarea)
    return pl.concat(
        (
            _calculate_maz_external_od(
                daysim_exit_trips, subarea_extraction_exit_od, "o", "d"
            ),
            _calculate_maz_external_od(
                daysim_enter_trips, subarea_extraction_enter_od, "d", "o"
            ),
        ),
        how="diagonal",
    )


def calculate_maz_internal_od(daysim_trips, subarea_extraction_od):
    internal = origin_in_subarea & destination_in_subarea
    daysim_internal_trips = daysim_trips.filter(internal)
    subarea_extraction_internal_od = subarea_extraction_od.filter(internal)
    return _calculate_maz_internal_od(
        daysim_internal_trips, subarea_extraction_internal_od
    )


def _calculate_maz_external_od(
    daysim_external_trips, subarea_extraction_external_od, internal, external
):
    """
    this is for calculating just trips entering or exiting the subarea
    internal: "o" or "d"
    external: "o" or "d"
    we take the subarea extraction ("_taz") volumes as canonical,
    and adjust them based on the Daysim ("_daysim_maz") volumes
    Note that the Cube results include visitor trips (which Daysim
    (residential trips only) doesn't) which we ignore in these calculations
    """
    if internal not in {"o", "d"}:
        raise ValueError(
            'internal should be either "o" (for trips exiting the subarea) or '
            '"d" (for trips entering the subarea).'
        )
    if external not in {"o", "d"}:
        raise ValueError(
            'internal should be either "o" (for trips entering the subarea) or'
            ' "d" (for trips exiting the subarea).'
        )
    if internal == external:
        raise ValueError('internal and external cannot both be "o" or "d".')

    # sum up all auto person-trips (sov, hov2/3+, tnc) at the MAZ and TAZ level
    # to estimate TRK and COM trips to/from each MAZ later
    calculate_auto_tot = pl.sum_horizontal(
        "sov", "hov2", "hov3+", "tnc"
    ).alias("auto_tot")
    daysim_external_trips = daysim_external_trips.pivot(
        values="mode",
        index=[f"{internal}pcl", f"{internal}taz"],
        columns="mode",
        aggregate_function="len",
    ).with_columns(calculate_auto_tot)
    daysim_external_trips_taz_tot = (
        daysim_external_trips.group_by(f"{internal}taz")
        .sum()
        .drop(f"{internal}pcl")
        .with_columns(calculate_auto_tot)
        .select(pl.all().name.suffix("_daysim_taz_tot"))
    )

    external_veh_trips = (
        daysim_external_trips.select(pl.all().name.suffix("_daysim_maz"))
        .join(
            # cross-join: get all combos of internal MAZ x external stations
            subarea_extraction_external_od.select(
                "otaz",
                "dtaz",
                # convert residential vehicle-trips to person-trips
                pl.col("DA").alias("sov"),
                (pl.col("SR2") * 2).alias("hov2"),
                (pl.col("SR3") * 3.5).alias("hov3+"),
                # tnc: tnc person-trips: only for calculating  auto_tot later
                (
                    pl.col("TNC1") + pl.col("TNC2") * 2 + pl.col("TNC3") * 3
                ).alias("tnc"),
                # TNC: TNC vehicle-trips: since we ultimatly want vehicle-trips
                (pl.col("TNC1") + pl.col("TNC2") + pl.col("TNC3")).alias(
                    "TNC"
                ),
                # keep truck/commercial vehicle-trips as is
                "TRK",
                "COM",
            ).select(pl.all().name.suffix("_taz")),
            how="cross",
            # then filter for only the row where the internal o/dtaz matches
            # NOTE The whole row (from the subarea extraction) would be lost if
            # there's no trips to/from that TAZ in the Daysim trip list.
        )
        .filter(
            pl.col(f"{internal}taz_daysim_maz") == pl.col(f"{internal}taz_taz")
        )
        .join(
            daysim_external_trips_taz_tot,
            left_on=f"{internal}taz_daysim_maz",
            right_on=f"{internal}taz_daysim_taz_tot",
        )
        .select(
            pl.col(f"{internal}pcl_daysim_maz").alias(f"{internal}maz"),
            pl.col(f"{internal}taz_taz").alias(f"{internal}taz"),
            pl.col(f"{external}taz_taz").alias(
                f"{external}maz"
            ),  # treating external stations as a "maz"
            # Estimate the number of person- (sov, hov2/3+, tnc) or vehicle-
            # (TRK, COM) trips from each origin to each external node then
            # convert from person- to vehicle-trips.
            # NOTE Some trips present in the subarea extraction results may be
            # lost if there are no trips to/from that TAZ in that vehicle
            # subtype in daysim, since xxx_daysim_taz_tot and xxx_daysim_maz
            # would both be 0/null.
            (
                pl.col("sov_taz")
                * pl.col("sov_daysim_maz")
                / pl.col("sov_daysim_taz_tot")
            )
            .fill_null(0)
            .alias("DA"),
            (
                pl.col("hov2_taz")
                * pl.col("hov2_daysim_maz")
                / pl.col("hov2_daysim_taz_tot")
                / 2
            )
            .fill_null(0)
            .alias("SR2"),
            (
                pl.col("hov3+_taz")
                * pl.col("hov3+_daysim_maz")
                / pl.col("hov3+_daysim_taz_tot")
                / 3.5
            )
            .fill_null(0)
            .alias("SR3"),
            (
                pl.col("TNC_taz")
                * pl.col("tnc_daysim_maz")
                / pl.col("tnc_daysim_taz_tot")
            )
            .fill_null(0)
            .alias("TNC"),
            # ASSUMPTION: distribute TRK/COM trips to MAZs based on residential
            # trips this may bias TRK/COM trips away from commercial corridors
            # to residential areas
            # FUTURE TODO MAYBE: using maz-level employment to distribute trips
            # from TAZ to MAZ-level, as employment is a big driver of COM and
            # TRK trips
            (
                pl.col("TRK_taz")
                * pl.col("auto_tot_daysim_maz")
                / pl.col("auto_tot_daysim_taz_tot")
            )
            .fill_null(0)
            .alias("TRK"),
            (
                pl.col("COM_taz")
                * pl.col("auto_tot_daysim_maz")
                / pl.col("auto_tot_daysim_taz_tot")
            )
            .fill_null(0)
            .alias("COM"),
        )
    )
    return external_veh_trips


def _calculate_maz_internal_od(
    daysim_internal_trips, subarea_extraction_internal_od
):
    """
    this is for calculating just trips internal to the subarea
    we take the subarea extraction ("_taz") volumes as canonical,
    and adjust them based on the Daysim ("_daysim_maz") volumes

    Note that the Cube results include visitor trips (which Daysim (residential
    trips only) doesn't) which we ignore in these calculations
    """
    # sum up all auto person-trips (sov, hov2/3+, tnc) at the MAZ and TAZ level
    # to estimate TRK and COM trips to/from each MAZ later
    calculate_auto_tot = pl.sum_horizontal(
        "sov", "hov2", "hov3+", "tnc"
    ).alias("auto_tot")

    daysim_internal_trips = daysim_internal_trips.pivot(
        values="mode",
        index=["opcl", "otaz", "dpcl", "dtaz"],
        columns="mode",
        aggregate_function="len",
    ).with_columns(calculate_auto_tot)

    daysim_internal_trips_taz_tot = (
        daysim_internal_trips.group_by("otaz", "dtaz")
        .sum()
        .drop("opcl", "dpcl")
        .with_columns(calculate_auto_tot)
        .select(pl.all().name.suffix("_daysim_taz_tot"))
    )

    # tnc & TNC calculations
    subarea_extraction_internal_od = subarea_extraction_internal_od.with_columns(
        (pl.col("TNC1")).alias("tnc1"),
        (pl.col("TNC2") * 2).alias("tnc2"),
        (pl.col("TNC3") * 3).alias("tnc3"),
    ).with_columns(
        # tnc: tnc person-trips: also for calculating auto_tot later
        (pl.col("tnc1") + pl.col("tnc2") + pl.col("tnc3")).alias("tnc")
        # # TNC: TNC vehicle-trips: since we ultimatly want vehicle-trips
        # (pl.col("TNC1") + pl.col("TNC2") + pl.col("TNC3")).alias("TNC"),
    )

    # for Daysim MAZ-pairs within TAZ-pairs that don't have any Cube/CHAMP
    # trips, estimate the TNC vehicle-trips from Daysim person-trips using the
    # conversion factor for tnc person-trips to TNC vehicle-trips for the whole
    # subarea
    subarea_tnc_to_TNC = (
        subarea_extraction_internal_od.select("tnc1", "tnc2", "tnc3", "tnc")
        .sum()
        .select(
            # calculate fractions of tnc1/2/3 person-trips out of all tnc
            # person-trips within the whole subarea
            (pl.col("tnc1") / pl.col("tnc")).alias("subarea_tnc1_fraction"),
            (pl.col("tnc2") / pl.col("tnc")).alias("subarea_tnc2_fraction"),
            (pl.col("tnc3") / pl.col("tnc")).alias("subarea_tnc3_fraction"),
        )
        .select(
            (
                pl.col("subarea_tnc1_fraction")
                + pl.col("subarea_tnc2_fraction") / 2
                + pl.col("subarea_tnc3_fraction") / 3
            ).alias("subarea_tnc_to_TNC")
        )
    )
    internal_veh_trips = (
        daysim_internal_trips.select(pl.all().name.suffix("_daysim_maz"))
        .join(
            # cross-join: get all combos of internal MAZ x internal TAZ
            subarea_extraction_internal_od.select(
                "otaz",
                "dtaz",
                # # convert residential vehicle-trips to person-trips
                # For internal sov/hov2/hov3+ (DA/SR2/SR3) trips,
                # use Daysim trip numbers directly
                "tnc1",
                "tnc2",
                "tnc3",
                "tnc",
                # keep truck/commercial vehicle-trips as is
                "TRK",
                "COM",
            ).select(pl.all().name.suffix("_taz")),
            how="cross",
            # then filter for only the rows where the o/dtaz match
        )
        .filter(
            # NOTE The whole row (from the subarea extraction) would be lost if
            # there's no trips to/from that TAZ in the Daysim trip list.
            (pl.col("otaz_daysim_maz") == pl.col("otaz_taz"))
            & (pl.col("dtaz_daysim_maz") == pl.col("dtaz_taz"))
        )
        .join(
            daysim_internal_trips_taz_tot,
            left_on=["otaz_daysim_maz", "dtaz_daysim_maz"],
            right_on=["otaz_daysim_taz_tot", "dtaz_daysim_taz_tot"],
        )
        .select(
            pl.col("opcl_daysim_maz").alias("omaz"),
            pl.col("dpcl_daysim_maz").alias("dmaz"),
            pl.col("otaz_taz").alias("otaz"),
            pl.col("dtaz_taz").alias("dtaz"),
            # Estimate the number of person- (sov, hov2/3+, tnc) or vehicle-
            # (TRK, COM) trips from each origin to each external node then
            # convert from person- to vehicle-trips.
            # For internal sov/hov2/hov3+ trips, just use the Daysim trips OD
            # numbers directly, even though there's slight discrepancies
            # between Daysim and final Cube/CHAMP numbers.
            (pl.col("sov_daysim_maz"))
            .fill_null(0)
            .cast(float)
            .alias("DA"),  # cast(float) for pl.concat later
            (pl.col("hov2_daysim_maz") / 2).fill_null(0).alias("SR2"),
            (pl.col("hov3+_daysim_maz") / 3.5).fill_null(0).alias("SR3"),
            # For internal tnc trips, we start from Daysim person-volumes, and
            # estimate TNC1/2/3 based on CHAMP/Cube volumes for the TAZ OD pair
            pl.when(pl.col("tnc_taz") > 0)
            .then(
                (
                    pl.col("tnc_daysim_maz")
                    * pl.col("tnc1_taz")
                    / pl.col("tnc_taz")
                )
                + (  # estimated_tnc1_daysim_maz
                    pl.col("tnc_daysim_maz")
                    * pl.col("tnc2_taz")
                    / pl.col("tnc_taz")
                    / 2
                )
                + (  # estimated_tnc2_daysim_maz / 2
                    pl.col("tnc_daysim_maz")
                    * pl.col("tnc3_taz")
                    / pl.col("tnc_taz")
                    / 3
                )  # estimated_tnc3_daysim_maz / 3
            )
            .otherwise(pl.col("tnc_daysim_maz") * subarea_tnc_to_TNC)
            .fill_null(0)
            .alias("TNC"),
            # ASSUMPTION: distribute TRK/COM trips to MAZs based on residential
            # trips this may bias TRK/COM trips away from commercial corridors
            # to residential areas
            # NOTE Some trips present in the subarea extraction results may be
            # lost if there are no sov/hov2/hov3+/tnc trips to/from that TAZ in
            # daysim, since auto_tot_daysim_maz and auto_tot_daysim_taz_tot
            # would both be 0/null.
            # FUTURE TODO MAYBE: using maz-level employment to distribute trips
            # from TAZ to MAZ-level, as employment is a big driver of COM and
            # TRK trips
            (
                pl.col("TRK_taz")
                * pl.col("auto_tot_daysim_maz")
                / pl.col("auto_tot_daysim_taz_tot")
            )
            .fill_null(0)
            .alias("TRK"),
            (
                pl.col("COM_taz")
                * pl.col("auto_tot_daysim_maz")
                / pl.col("auto_tot_daysim_taz_tot")
            )
            .fill_null(0)
            .alias("COM"),
        )
    )
    return internal_veh_trips

In [9]:
veh_trips_maz_od = pl.concat(
    (
        calculate_maz_internal_od(daysim_trips, subarea_extraction_od),
        calculate_maz_external_od(daysim_trips, subarea_extraction_od),
        external_to_external_od,
    ),
    how="diagonal",
).select(
    "omaz", "dmaz", "otaz", "dtaz", "DA", "SR2", "SR3", "TNC", "TRK", "COM"
)
veh_trips_maz_od.write_csv(out_filepath)